# Creo una db duckdb con unos puntos seleccionados

Aemet facilita los datos de pcp, tmax y tmin diarios en una rejilla de 5km: https://www.aemet.es/es/serviciosclimaticos/cambio_climat/datos_diarios?w=2

EStos datos se suministran en ficheros txt comprimidos con gzip y agrupados en ficheros tar. En otro nb he pasado los 3 ficheros de datos climáticos de Aemet a formato parquet (pcp, tmin, tmax). Los ficheros parquet presentan ventajas de acceso con respecto a los originales que se descargan de Aemet, aunque ocupan un poco más.

Para el trabajo de un proyecto, presenta ventajas el disponer de los datos de la zona de interés en un formato de db relacional, con una tabla de puntos y otra con los datos climáticos (pcp, tmax y tmin).

En este nb se extraen de los ficheros parquet los datos de los puntos seleccionados, se les cambia la estructura a un formato relacional y se guardan en una db duckdb


In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import duckdb
import duckdb_utilities as ducku
import pandas as pd
from pathlib import Path 
import pyarrow.parquet as pq

import sys
dpath = Path(r'E:\DB\aemet\rejilla5km')
success = True
if dpath.exists() and dpath.is_dir():
    if dpath not in sys.path:
        sys.path.append(str(dpath))
        print(f'Se ha añadido {dpath} to sys.path')
    else:
        print(f'{dpath} ya estaba incluido en sys.path')
else:
    print(f'{dpath} no existe o no es un directorio')
    success = False
if success:
    import Aemet_create_duckdb_db as aemet


Se ha añadido E:\DB\aemet\rejilla5km to sys.path


## DB rocio

El nombre rocio significa Rejilla Observacional Con Interpolación Óptima (ROCIO) y ha sido escogido por Aemet para hacer referencia a los datos diarios de pcp, tmax y tmin en una rejilla de 5 km. Canarias va aparte y su rejilla es de 2.5 km.

Se pueden crear dos tipos de db según se organicen los datos en tablas

* Una db con 3 tablas: pcp, tmin y tmax. La creación de estas 3 tablas es obligatoria.
    * pcp, con columnas: fid, fecha, pcp
    * tmax, con columnas: fid, fecha, tmax
    * tmin, con columnas: fid, fecha, tmin
* Una bd con una sola tabla, rocio, que tiene como columnas fid, fecha, pcp, tmin, tmax

Además de las tablas descritas para pcp, tmax y tmin, siempre hay una tabla adicional, puntos, con las coordenadas y z de los puntos de la rejilla. 

La tabla puntos está en coordenadas epsg 4258 pero también se han guardados las coordenadas proyectadas x y en epsg 25830; se pueden actualizar las coordenadas proyectadas (x, y) de los puntos en las zonas 25829 o 25831 transformando sus coordenadas y anotando el epsg; en este momento todos los puntos están en 25830 

In [3]:
dbpath = r'E:\LSGB\20240603_Los_Santos\db\aemet_rocio.duckdb'

### Puntos seleccionados del grid

En el ejemplo a continuación los puntos se han obtenido como una operación de intersección en QGIS y exportado a un fichero CSV. Leo el fichero CSV y guardo la columna fid (identificados del punto) como una lista.

En otros casos con menor puntos, puedo crear directamente la lista de puntos.


In [4]:
fpuntos_envolvente = r'E:\LSGB\20240603_Los_Santos\qgis\layers\grid_aemet_cs_envolvente.csv'

df = pd.read_csv(fpuntos_envolvente)
print(df.info())

ptos_env = list(df['id'])

cols_env = [0,] + ptos_env

len(ptos_env)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       144 non-null    float64
 1   Y       144 non-null    float64
 2   id      144 non-null    int64  
 3   z       144 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 4.6 KB
None


144

### Insertar los puntos de la rejilla en la db

Se pueden insrtar todos o solo los seleccionados, que son los únicos que van a tener datos de pcp, tmax y tmin. En este caso los voy a insertar todos

In [25]:
def get_info_parquet_file(parquet_file):
    """ 
    Return the name and type of the columns of a parque file

    Parameters
    ---------
    parquet_file : str
    """
    table = pq.read_table(parquet_file)
    with duckdb.connect() as con:
        con.register('parquet_table', table)
        schema = con.execute('DESCRIBE parquet_table').df()
        print(schema,'\n')
        df = con.execute('select * from parquet_table limit 5').df()
        print('head(5)')
        print(df.head())    
        df = con.execute('select count(*) as rows from parquet_table').df()
        print(df.head())    

In [26]:
fpath = r'E:\DB\aemet\rejilla5km\parquet\rejilla.parquet'

get_info_parquet_file(fpath)


  column_name column_type null   key default extra
0          id     INTEGER  YES  None    None  None
1         lon       FLOAT  YES  None    None  None
2         lat       FLOAT  YES  None    None  None
3           z       FLOAT  YES  None    None  None
4           x       FLOAT  YES  None    None  None
5           y       FLOAT  YES  None    None  None
6        epsg     VARCHAR  YES  None    None  None 

head(5)
   id    lon        lat      z             x           y   epsg
0   1 -2.948  35.306999  102.0  504727.28125  3907090.25  25830
1   2 -5.323  35.897999   58.0  290346.15625  3975128.00  25830
2   3 -5.630  36.049000  124.0  263085.28125  3992584.50  25830
3   4 -5.568  36.049000  186.0  268671.50000  3992435.25  25830
4   5 -5.507  36.049000  163.0  274167.56250  3992291.75  25830
    rows
0  16156


In [27]:
query = f"SELECT * FROM read_parquet('{fpath}')"
df = duckdb.query(query).df()
df.head()

,id,lon,lat,z,x,y,epsg
0,1,-2.948,35.306999,102.0,504727.28125,3907090.25,25830
1,2,-5.323,35.897999,58.0,290346.15625,3975128.00,25830
2,3,-5.630,36.049000,124.0,263085.28125,3992584.50,25830
3,4,-5.568,36.049000,186.0,268671.50000,3992435.25,25830
4,5,-5.507,36.049000,163.0,274167.56250,3992291.75,25830


In [28]:
create_table = \
f""" 
create table if not exists puntos
(
    fid integer primary key,
    lon real,
    lat real,
    z   real,
    x   real,
    y   real,
    epsg varchar
)
"""
query = f"INSERT INTO puntos SELECT * FROM df"

with duckdb.connect(dbpath) as con:
    con.execute(create_table)
    con.execute(query)
    con.commit()
    rows = con.execute('select * from puntos limit 5').df()
    print(rows)
    row = con.execute('select count(*) as rows from puntos').df()
    print(row)


   fid    lon        lat      z             x           y   epsg
0    1 -2.948  35.306999  102.0  504727.28125  3907090.25  25830
1    2 -5.323  35.897999   58.0  290346.15625  3975128.00  25830
2    3 -5.630  36.049000  124.0  263085.28125  3992584.50  25830
3    4 -5.568  36.049000  186.0  268671.50000  3992435.25  25830
4    5 -5.507  36.049000  163.0  274167.56250  3992291.75  25830
    rows
0  16156


### Insertar datos de pcp, tmin y tmax

In [21]:
# 'pcp, 'tmin, 'tmax'
var = 'tmin'

files = {
    'pcp':  r'E:\DB\aemet\rejilla5km\parquet\pcp.parquet',
    'tmax': r'E:\DB\aemet\rejilla5km\parquet\tmax.parquet',
    'tmin': r'E:\DB\aemet\rejilla5km\parquet\tmin.parquet'
}

if var not in files.keys():
    raise ValueError('Error al asignar var')

if files['pcp'] == files['tmax'] or files['pcp'] == files['tmin'] or files['tmax'] == files['tmin']:
    raise ValueError('No pude haber valores repetidos en files')

if var not in files[var]:
    raise ValueError(f'El fichero de Aemet debe contener {var}')

df = aemet.read_parquet_columns(files[var], cols_env)
# print(df.head())

up_df = aemet.melt_rocio_data(df)
# print(up_df.info())

create_table = \
f""" 
create table if not exists {var}
(
    fid integer,
    fecha date,
    {var} real,
    primary key(fid, fecha)
)
"""
query = f"INSERT INTO {var} SELECT * FROM up_df"

with duckdb.connect(dbpath) as con:
    con.execute(create_table)
    con.execute(query)
    con.commit()

    cols_df = ducku.get_columns(dbpath, var)
    print(cols_df)

    row = con.execute(f'select count(*) from {var}').df()
    print(row)
    
    rows = con.execute(f'select * from {var} limit 5').df()

rows.head()



           0  4178  4179  4180  4181  4182  4183  4184  4185  4186  ...  5413  \
0 1951-01-01  0.62  0.87  1.18  1.18  0.90  0.71  0.56  0.31  0.18  ...  0.46   
1 1951-01-02  1.41  1.60  1.95  2.13  1.91  1.66  1.29  0.88  0.60  ...  2.13   
2 1951-01-03  0.11  0.36  0.74  0.86  0.67  0.49  0.21 -0.14 -0.29  ...  0.14   
3 1951-01-04 -1.18 -0.84 -0.50 -0.53 -0.81 -0.90 -0.93 -1.03 -0.96  ... -1.03   
4 1951-01-05 -2.16 -1.81 -1.60 -1.88 -2.31 -2.35 -2.19 -2.06 -1.78  ... -2.85   

   5414  5415  5416  5417  5418  5419  5420  5421  5422  
0  0.68  1.25  2.00  2.87  3.75  4.03  4.31  4.06  4.28  
1  2.23  2.70  3.54  4.57  5.60  6.01  6.41  6.26  6.45  
2  0.27  0.83  1.64  2.67  3.74  4.27  4.83  4.83  5.11  
3 -0.71 -0.09  0.69  1.60  2.57  2.97  3.38  3.32  3.63  
4 -2.41 -1.69 -1.06 -0.41  0.37  0.59  0.90  0.81  1.12  

[5 rows x 145 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786912 entries, 0 to 3786911
Data columns (total 3 columns):
 #   Column  Dtype         
-

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  column_name data_type
0         fid   INTEGER
1       fecha      DATE
2        tmin     FLOAT
   count_star()
0       3786912


,fid,fecha,tmin
0,4178,1951-01-01,0.62
1,4178,1951-01-02,1.41
2,4178,1951-01-03,0.11
3,4178,1951-01-04,-1.18
4,4178,1951-01-05,-2.16


In [29]:
ducku.get_tables(dbpath)

,table_name
0,pcp
1,puntos
2,tmax
3,tmin


In [36]:
tables = ducku.get_tables(dbpath)['table_name'].tolist()
with duckdb.connect(dbpath) as con:
    for table in tables:
        print(table)
        print(ducku.get_columns(dbpath, table))
        row = con.execute(f'select count(*) as rows from {table}').df()
        print(row)


pcp
  column_name data_type
0         fid   INTEGER
1       fecha      DATE
2         pcp     FLOAT
      rows
0  3786912
puntos
  column_name data_type
0         fid   INTEGER
1         lon     FLOAT
2         lat     FLOAT
3           z     FLOAT
4           x     FLOAT
5           y     FLOAT
6        epsg   VARCHAR
    rows
0  16156
tmax
  column_name data_type
0         fid   INTEGER
1       fecha      DATE
2        tmax     FLOAT
      rows
0  3786912
tmin
  column_name data_type
0         fid   INTEGER
1       fecha      DATE
2        tmin     FLOAT
      rows
0  3786912


## Merge tables

Esto es opcional, antes de hacer nada hago una copia de la db con las 3 tablas.

Ahora voy a pasar las tablas de pcp, tmax, tmin a una sola tabla rocio

In [37]:
create_table_rocio = \
f""" 
create table if not exists rocio(
    fid varchar,
    fecha date,
    pcp real, 
    tmin real,
    tmax real,
    primary key(fid, fecha)
)
"""

insert_pcp = \
""" 
insert into rocio (fid, fecha, pcp)
select fid, fecha, pcp from pcp order by fid,fecha
"""
with duckdb.connect(dbpath) as con:
    con.execute(create_table_rocio)
    con.execute(insert_pcp)
con.commit()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [40]:
update_template = \
""" 
update rocio
set {} = t.{}
from {} t
where rocio.fid = t.fid and rocio.fecha = t.fecha
"""
with duckdb.connect(dbpath) as con:
    for var in ('tmax', 'tmin'):
        update_stm = update_template.format(var, var, var)
        con.execute(update_stm)
    con.commit()
    rocio_df = con.execute('select * from rocio order by fid, fecha limit 5').df()
    row = con.execute('select count(*) as row from rocio').df()
print('head(5)')
print(rocio_df)
print(row)


head(5)
    fid      fecha   pcp  tmin   tmax
0  4178 1951-01-01  0.00  0.62   9.02
1  4178 1951-01-02  0.91  1.41   8.90
2  4178 1951-01-03  1.40  0.11   7.76
3  4178 1951-01-04  0.03 -1.18   9.13
4  4178 1951-01-05  0.00 -2.16  10.17
       row
0  3786912


### Borrar las tablas de partida y recuperar el espacio

In [41]:
drop_table_template = 'drop table if exists {}'
with duckdb.connect(dbpath) as con:
    for var in ('pcp', 'tmax', 'tmin'):
        drop_table_stm = drop_table_template.format(var)
    con.execute("VACUUM")
    con.execute("CHECKPOINT")
 

## Resumen y conclusiones

* La rejilla de 5 km de Aemet tiene algo más de 16,000 puntos
* Se han seleccionado unos 140 puntos y se han creado 2 db
* La db con los datos clmáticos en 3 tablas ocupa 178.76 Mb
* La db con los datos climáticos en una tabla, rocio, ocupa 115.76 Mb
* Por la comodidad trabajar con una tabla y el menor tamaño de la db, se selecciona como la mejor de las 2 opciones  